In [12]:
import numpy as np
import pandas as pd

In [2]:
df1 = pd.read_excel('pwt100.xlsx', sheet_name = 'Data')
cols = ['countrycode', 'country', 'currency_unit', 'year', 'labsh']
labsh = df1[cols]

In [3]:
df1

# Select column
# Filter year, country
# Long format

,countrycode,country,currency_unit,year,rgdpe,rgdpo,pop,emp,avh,hc,...,csh_x,csh_m,csh_r,pl_c,pl_i,pl_g,pl_x,pl_m,pl_n,pl_k
0,ABW,Aruba,Aruban Guilder,1950,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABW,Aruba,Aruban Guilder,1951,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABW,Aruba,Aruban Guilder,1952,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ABW,Aruba,Aruban Guilder,1953,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABW,Aruba,Aruban Guilder,1954,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12805,ZWE,Zimbabwe,US Dollar,2015,40141.617188,39798.644531,13.814629,6.393752,NaN,2.584653,...,0.140172,-0.287693,-0.051930,0.479228,0.651287,0.541446,0.616689,0.533235,0.422764,1.533909
12806,ZWE,Zimbabwe,US Dollar,2016,41875.203125,40963.191406,14.030331,6.504374,NaN,2.616257,...,0.131920,-0.251232,-0.016258,0.470640,0.651027,0.539631,0.619789,0.519718,0.416510,1.491724
12807,ZWE,Zimbabwe,US Dollar,2017,44672.175781,44316.742188,14.236595,6.611773,NaN,2.648248,...,0.126722,-0.202827,-0.039897,0.473560,0.639560,0.519956,0.619739,0.552042,0.415592,1.514525
12808,ZWE,Zimbabwe,US Dollar,2018,44325.109375,43420.898438,14.438802,6.714952,NaN,2.680630,...,0.144485,-0.263658,-0.020791,0.543757,0.655473,0.529867,0.641361,0.561526,0.425143,1.590120


In [4]:
df2 = pd.read_csv('API_NY.GDP.PCAP.PP.CD_DS2_en_csv_v2_4150885.csv')

In [5]:
df2

#  Wide - long
# Filter indicator, country, year

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Aruba,ABW,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,33567.550020,36829.032770,36779.429430,37311.750320,37585.025080,38897.122670,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3237.870658,3365.682977,3503.699383,3533.230819,3627.294348,3702.390685,3791.875407,3842.578511,3684.562623,NaN
2,Afghanistan,AFG,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1914.774228,2015.514775,2069.424022,2087.305323,1981.118069,2058.400221,2082.635648,2152.366489,2078.648615,NaN
3,Africa Western and Central,AFW,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3772.323802,4013.196523,4208.731780,4139.323364,4069.005667,4115.645480,4231.815774,4335.199547,4242.114489,NaN
4,Angola,AGO,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,7412.967035,7682.475386,8179.297828,7337.569822,7103.226431,7310.896589,7099.971958,6952.419362,6445.432873,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,8039.207976,8383.397311,8649.449499,9357.135136,10007.689230,10436.168850,11013.397080,11797.119320,11293.783760,NaN
262,"Yemen, Rep.",YEM,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3491.204610,3688.519409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263,South Africa,ZAF,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,13215.708680,13606.942900,13602.102640,13701.945390,13748.461420,13860.270170,14209.093830,14289.760000,13360.574010,NaN
264,Zambia,ZMB,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3422.619937,3578.847055,3539.436390,3430.400287,3404.673990,3485.021780,3606.097632,3617.200014,3457.610102,NaN


In [6]:
df3 = pd.read_csv('API_BN.CAB.XOKA.GD.ZS_DS2_en_csv_v2_4150849.csv')

In [7]:
df3
#  Wide - long
# Filter indicator, country, year

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Aruba,ABW,Current account balance (% of GDP),BN.CAB.XOKA.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,3.583866,-12.142913,-4.855105,3.735864,4.557757,0.954621,-0.460233,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,Current account balance (% of GDP),BN.CAB.XOKA.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,Current account balance (% of GDP),BN.CAB.XOKA.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,-25.870698,-25.290073,-15.772421,-21.912669,-14.950203,-18.955948,-21.585267,-20.170458,-15.593120,NaN
3,Africa Western and Central,AFW,Current account balance (% of GDP),BN.CAB.XOKA.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,AGO,Current account balance (% of GDP),BN.CAB.XOKA.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,10.808963,5.957924,-2.571863,-8.841139,-3.050908,-0.518218,7.303770,5.745422,1.493624,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,Current account balance (% of GDP),BN.CAB.XOKA.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,-6.160290,-3.580378,-7.090254,-8.694464,-7.934241,-5.263446,-7.621979,-5.661265,-7.011942,NaN
262,"Yemen, Rep.",YEM,Current account balance (% of GDP),BN.CAB.XOKA.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.945211,-3.786913,-3.442753,-7.129267,-7.724024,NaN,NaN,NaN,NaN,NaN
263,South Africa,ZAF,Current account balance (% of GDP),BN.CAB.XOKA.GD.ZS,0.646831,3.388938,5.057661,2.065072,-1.551957,-4.557806,...,-4.683736,-5.338373,-4.812433,-4.310287,-2.633615,-2.374368,-3.035027,-2.594519,2.026643,NaN
264,Zambia,ZMB,Current account balance (% of GDP),BN.CAB.XOKA.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,5.378985,-0.575956,-1.427094,-3.612272,-4.551688,-1.681271,-1.301163,0.603691,13.432721,NaN


In [8]:
df4 = pd.read_csv('DP_LIVE_28052022104927583.csv')

In [9]:
df4

# Select country, measure, time

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUS,GGNLEND,TOT,PC_GDP,A,2000,-1.292779,NaN
1,AUS,GGNLEND,TOT,PC_GDP,A,2001,-0.743403,NaN
2,AUS,GGNLEND,TOT,PC_GDP,A,2002,0.706782,NaN
3,AUS,GGNLEND,TOT,PC_GDP,A,2003,0.794190,NaN
4,AUS,GGNLEND,TOT,PC_GDP,A,2004,0.927764,NaN
...,...,...,...,...,...,...,...,...
916,IDN,GGNLEND,TOT,PC_GDP,A,2015,-2.553798,NaN
917,IDN,GGNLEND,TOT,PC_GDP,A,2016,-2.375840,NaN
918,IDN,GGNLEND,TOT,PC_GDP,A,2017,-1.956537,NaN
919,IDN,GGNLEND,TOT,PC_GDP,A,2018,-1.235415,NaN


In [10]:
df5 = pd.read_csv('DP_LIVE_28052022104919240.csv')

In [11]:
df5

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUS,GGNLEND,TOT,PC_GDP,A,1970,-1.656802,NaN
1,AUS,GGNLEND,TOT,PC_GDP,A,1971,-1.627166,NaN
2,AUS,GGNLEND,TOT,PC_GDP,A,1972,-2.095081,NaN
3,AUS,GGNLEND,TOT,PC_GDP,A,1973,-0.959022,NaN
4,AUS,GGNLEND,TOT,PC_GDP,A,1974,-3.598636,NaN
...,...,...,...,...,...,...,...,...
1236,IDN,GGNLEND,TOT,PC_GDP,A,2015,-2.553798,NaN
1237,IDN,GGNLEND,TOT,PC_GDP,A,2016,-2.375840,NaN
1238,IDN,GGNLEND,TOT,PC_GDP,A,2017,-1.956537,NaN
1239,IDN,GGNLEND,TOT,PC_GDP,A,2018,-1.235415,NaN
